In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import tensorflow as tf
import keras
from keras.layers import Dense,Conv2D,MaxPool2D,BatchNormalization,Conv2DTranspose,Dropout,LeakyReLU
from keras.layers import LSTM,Bidirectional,Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import regularizers
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
import pandas as pd
import numpy as np

In [3]:
!wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt -O sonnets.txt

--2022-01-13 11:00:23--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘sonnets.txt’

sonnets.txt         100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2022-01-13 11:00:24 (147 MB/s) - ‘sonnets.txt’ saved [93578/93578]



**Reading Data**

In [4]:
data = open('/content/sonnets.txt').read()

In [5]:
corpus = data.lower().split('\n')

In [6]:
print(corpus)

['from fairest creatures we desire increase,', "that thereby beauty's rose might never die,", 'but as the riper should by time decease,', 'his tender heir might bear his memory:', 'but thou, contracted to thine own bright eyes,', "feed'st thy light'st flame with self-substantial fuel,", 'making a famine where abundance lies,', 'thyself thy foe, to thy sweet self too cruel.', "thou that art now the world's fresh ornament", 'and only herald to the gaudy spring,', 'within thine own bud buriest thy content', 'and, tender churl, makest waste in niggarding.', 'pity the world, or else this glutton be,', "to eat the world's due, by the grave and thee.", 'when forty winters shall beseige thy brow,', "and dig deep trenches in thy beauty's field,", "thy youth's proud livery, so gazed on now,", "will be a tatter'd weed, of small worth held:", "then being ask'd where all thy beauty lies,", 'where all the treasure of thy lusty days,', 'to say, within thine own deep-sunken eyes,', 'were an all-eating

**Tokenizer**

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

In [8]:
print(tokenizer)

In [9]:
total_words = len(tokenizer.word_index)+1

In [10]:
index = tokenizer.word_index

In [11]:
print(index)

{'and': 1, 'the': 2, 'to': 3, 'of': 4, 'my': 5, 'i': 6, 'in': 7, 'that': 8, 'thy': 9, 'thou': 10, 'with': 11, 'for': 12, 'is': 13, 'love': 14, 'not': 15, 'but': 16, 'a': 17, 'me': 18, 'thee': 19, 'so': 20, 'be': 21, 'as': 22, 'all': 23, 'you': 24, 'his': 25, 'which': 26, 'when': 27, 'it': 28, 'this': 29, 'by': 30, 'your': 31, 'doth': 32, 'do': 33, 'from': 34, 'on': 35, 'or': 36, 'no': 37, 'then': 38, 'have': 39, 'what': 40, 'are': 41, 'if': 42, 'more': 43, 'mine': 44, 'their': 45, 'shall': 46, 'sweet': 47, 'time': 48, 'will': 49, 'they': 50, 'beauty': 51, 'nor': 52, 'eyes': 53, 'art': 54, 'her': 55, 'heart': 56, 'yet': 57, 'o': 58, 'than': 59, 'can': 60, 'should': 61, 'thine': 62, 'now': 63, 'where': 64, 'make': 65, 'one': 66, 'hath': 67, 'he': 68, 'fair': 69, 'still': 70, 'how': 71, 'eye': 72, 'him': 73, 'like': 74, 'true': 75, 'see': 76, 'am': 77, 'she': 78, 'those': 79, 'though': 80, 'being': 81, 'some': 82, 'every': 83, 'such': 84, 'own': 85, 'were': 86, 'dost': 87, 'who': 88, 'liv

**Text to number**

In [12]:
token_list = tokenizer.texts_to_sequences(corpus)

In [13]:
print(token_list)

[[34, 417, 877, 166, 213, 517], [8, 878, 134, 351, 102, 156, 199], [16, 22, 2, 879, 61, 30, 48, 634], [25, 311, 635, 102, 200, 25, 278], [16, 10, 880, 3, 62, 85, 214, 53], [1372, 9, 1373, 636, 11, 122, 1374, 1375], [201, 17, 1376, 64, 518, 202], [118, 9, 1377, 3, 9, 47, 122, 135, 279], [10, 8, 54, 63, 2, 418, 312, 419], [1, 352, 1378, 3, 2, 1379, 420], [215, 62, 85, 881, 1380, 9, 882], [1, 311, 883, 884, 313, 7, 1381], [257, 2, 94, 36, 353, 29, 1382, 21], [3, 637, 2, 418, 354, 30, 2, 638, 1, 19], [27, 1383, 885, 46, 1384, 9, 280], [1, 1385, 281, 1386, 7, 9, 134, 1387], [9, 1388, 179, 1389, 20, 1390, 35, 63], [49, 21, 17, 886, 639, 4, 887, 126, 888], [38, 81, 1391, 64, 23, 9, 51, 202], [64, 23, 2, 258, 4, 9, 889, 145], [3, 95, 215, 62, 85, 281, 1392, 53], [86, 146, 23, 1393, 236, 1, 1394, 96], [71, 136, 43, 96, 1395, 9, 134, 186], [42, 10, 1396, 640, 890, 69, 282, 4, 44], [46, 519, 5, 520, 1, 65, 5, 112, 314, 147], [1397, 25, 51, 30, 1398, 62], [29, 86, 3, 21, 98, 127, 27, 10, 54, 112],

**n_gram_seq**

In [14]:
token_list = []

for line in corpus:
  token_l = tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_l)):
    n_gram = token_l[:i+1]
    token_list.append(n_gram)

In [15]:
max_seq_length = max(len(seq) for seq in token_list)

**Padding**

In [16]:
input_sequences = np.array(pad_sequences(token_list,padding='pre'))

In [17]:
input_sequences

array([[   0,    0,    0, ...,    0,   34,  417],
       [   0,    0,    0, ...,   34,  417,  877],
       [   0,    0,    0, ...,  417,  877,  166],
       ...,
       [   0,    0,    0, ...,  493,  493, 3210],
       [   0,    0,    0, ...,  493, 3210,   15],
       [   0,    0,    0, ..., 3210,   15,   14]], dtype=int32)

In [18]:
xs,labels = input_sequences[:,:-1],input_sequences[:,-1]

In [19]:
ys = tf.keras.utils.to_categorical(labels,num_classes=total_words)

In [20]:
xs

array([[   0,    0,    0, ...,    0,    0,   34],
       [   0,    0,    0, ...,    0,   34,  417],
       [   0,    0,    0, ...,   34,  417,  877],
       ...,
       [   0,    0,    0, ..., 3209,  493,  493],
       [   0,    0,    0, ...,  493,  493, 3210],
       [   0,    0,    0, ...,  493, 3210,   15]], dtype=int32)

**Building LSTM** **model**

In [27]:
model = Sequential()
model.add(Embedding(total_words,100,input_length = max_seq_length-1))
model.add(Bidirectional(LSTM(150,return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(200,return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(150,return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(96)))
model.add(Dense(total_words//2, activation='relu'))
model.add(Dense(total_words, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 10, 100)           321100    
                                                                 
 bidirectional_4 (Bidirectio  (None, 10, 300)          301200    
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 10, 300)           0         
                                                                 
 bidirectional_5 (Bidirectio  (None, 10, 400)          801600    
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 10, 400)           0         
                                                                 
 bidirectional_6 (Bidirectio  (None, 10, 300)         

**Training**

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(xs,ys,epochs=150,verbose=1)

Epoch 1/150
484/484 [==============================] - 38s 51ms/step - loss: 6.8307
Epoch 2/150
484/484 [==============================] - 25s 51ms/step - loss: 6.5999
Epoch 3/150
484/484 [==============================] - 25s 51ms/step - loss: 6.5713
Epoch 4/150
484/484 [==============================] - 25s 51ms/step - loss: 6.5412
Epoch 5/150
484/484 [==============================] - 25s 51ms/step - loss: 6.4655
Epoch 6/150
484/484 [==============================] - 25s 51ms/step - loss: 6.4028
Epoch 7/150
484/484 [==============================] - 25s 51ms/step - loss: 6.3149
Epoch 8/150
484/484 [==============================] - 25s 51ms/step - loss: 6.2479
Epoch 9/150
484/484 [==============================] - 25s 51ms/step - loss: 6.1898
Epoch 10/150
484/484 [==============================] - 25s 51ms/step - loss: 6.1373
Epoch 11/150
484/484 [==============================] - 25s 51ms/step - loss: 6.0912
Epoch 12/150
484/484 [==============================] - 25s 51ms/step - lo

In [22]:
import pickle

In [33]:
from keras.models import load_model
new_model = load_model('/content/drive/MyDrive/Shakespearre_model/Model_150_epochs/')

In [34]:
# new_model.compile(loss='categorical_crossentropy', optimizer='adam')
new_model.fit(xs,ys,epochs=150,verbose=1)

Epoch 1/150
484/484 [==============================] - 21s 18ms/step - loss: 1.4970
Epoch 2/150
484/484 [==============================] - 9s 18ms/step - loss: 1.4059
Epoch 3/150
484/484 [==============================] - 9s 18ms/step - loss: 1.3962
Epoch 4/150
484/484 [==============================] - 9s 18ms/step - loss: 1.3884
Epoch 5/150
484/484 [==============================] - 9s 18ms/step - loss: 1.3728
Epoch 6/150
484/484 [==============================] - 9s 18ms/step - loss: 1.3753
Epoch 7/150
484/484 [==============================] - 9s 18ms/step - loss: 1.3529
Epoch 8/150
484/484 [==============================] - 9s 18ms/step - loss: 1.3478
Epoch 9/150
484/484 [==============================] - 9s 18ms/step - loss: 1.3488
Epoch 10/150
484/484 [==============================] - 9s 18ms/step - loss: 1.3238
Epoch 11/150
484/484 [==============================] - 9s 18ms/step - loss: 1.3028
Epoch 12/150
484/484 [==============================] - 9s 18ms/step - loss: 1.3508


In [35]:
new_model.save('/content/drive/MyDrive/Shakespearre_model/Model_300_epochs')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Shakespearre_model/Model_300_epochs/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Shakespearre_model/Model_300_epochs/assets


In [ ]:
def generate_text(seed_text, next_words):
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_seq_length-1, padding='pre')
    predict_x=model.predict(token_list) 
    predicted=np.argmax(predict_x,axis=1)
    # predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += " " + output_word

  print(seed_text)
  return seed_text

In [ ]:
seed_text = 'Thyself thy foe'
next_words = 10
generated_text = generate_text(seed_text, next_words)

Thyself thy foe to thy sweet self prove bear sense burn sense burn


In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!mkdir Shakespearre_model

mkdir: cannot create directory ‘Shakespearre_model’: File exists


In [ ]:
%cd /content/drive/MyDrive/Shakespearre_model/

/content/drive/MyDrive/Shakespearre_model


In [ ]:
model.save('/content/drive/MyDrive/Shakespearre_model/Model_150_epochs')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Shakespearre_model/Model_150_epochs/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Shakespearre_model/Model_150_epochs/assets
